# IFPRS Data Visualization

This notebook visualizes the IFPRS (Integrated Fire Planning and Reporting System) data from the GDB file containing treatment activities across California.

In [ ]:
# Install necessary libraries
! pip install geopandas matplotlib seaborn folium contextily

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
import contextily as ctx
import numpy as np
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore', category=UserWarning)

# Set style for better visualizations
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

In [ ]:
# Load the IFPRS data from GDB
ifprs_gdb_path = "/tmp/IFPRS_2023_2025.gdb"
ifprs_layer_name = "IFPRS_enriched_20251210"

print(f"Loading layer '{ifprs_layer_name}' from {ifprs_gdb_path}...")

# Load the layer using geopandas
ifprs = gpd.read_file(ifprs_gdb_path, 
                      sql_dialect="OGRSQL", 
                      sql=f"SELECT *, OBJECTID FROM {ifprs_layer_name}")

print(f"Successfully loaded {len(ifprs)} records")
print(f"CRS: {ifprs.crs}")
print(f"Data columns: {len(ifprs.columns)}")

# Display basic info
print(f"\nFirst few rows:")
display(ifprs[['PROJECT_NAME', 'ADMIN_ORG_NAME', 'ACTIVITY_CAT', 'BROAD_VEGETATION_TYPE', 'ACTIVITY_QUANTITY']].head())

In [ ]:
# Transform to WGS84 (lat/long) for mapping
ifprs_wgs84 = ifprs.to_crs('EPSG:4326')
print(f"Transformed to WGS84: {ifprs_wgs84.crs}")

# Create centroids for point mapping (reproject to avoid warnings)
ifprs_proj = ifprs_wgs84.to_crs('EPSG:3857')  # Web Mercator for accurate centroids
ifprs_proj['centroid'] = ifprs_proj.geometry.centroid
ifprs_proj['longitude'] = ifprs_proj.centroid.x
ifprs_proj['latitude'] = ifprs_proj.centroid.y

# Transform centroids back to WGS84 for mapping
centroids_wgs84 = gpd.GeoDataFrame(ifprs_proj, geometry='centroid', crs='EPSG:3857').to_crs('EPSG:4326')
ifprs_wgs84['longitude'] = centroids_wgs84.geometry.x
ifprs_wgs84['latitude'] = centroids_wgs84.geometry.y

print(f"Coordinate ranges:")
print(f"Latitude: {ifprs_wgs84['latitude'].min():.4f} to {ifprs_wgs84['latitude'].max():.4f}")
print(f"Longitude: {ifprs_wgs84['longitude'].min():.4f} to {ifprs_wgs84['longitude'].max():.4f}")

## Interactive Map with IFPRS Activities

In [ ]:
# Create interactive map centered on California
center_lat = ifprs_wgs84['latitude'].mean()
center_lon = ifprs_wgs84['longitude'].mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=6, tiles='OpenStreetMap')

# Add color mapping for activity categories
activity_colors = {
    'MECH_HFR': 'red',
    'BENEFICIAL_FIRE': 'orange', 
    'NOT_DEFINED': 'gray',
    'GRAZING': 'green',
    'TREE_PLNTING': 'darkgreen',
    'WATSHD_IMPRV': 'blue'
}

# Add points to the map
for idx, row in ifprs_wgs84.iterrows():
    if pd.notna(row['latitude']) and pd.notna(row['longitude']):
        activity_cat = row['ACTIVITY_CAT']
        color = activity_colors.get(activity_cat, 'gray')
        
        popup_text = f"""
        <b>Project:</b> {row['PROJECT_NAME']}<br>
        <b>Admin Org:</b> {row['ADMIN_ORG_NAME']}<br>
        <b>Activity:</b> {row['ACTIVITY_CAT']}<br>
        <b>Vegetation:</b> {row['BROAD_VEGETATION_TYPE']}<br>
        <b>Quantity:</b> {row['ACTIVITY_QUANTITY']:.1f} {row['ACTIVITY_UOM']}<br>
        <b>Status:</b> {row['ACTIVITY_STATUS']}
        """
        
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            popup=folium.Popup(popup_text, max_width=300),
            color=color,
            fill=True,
            fillColor=color,
            fillOpacity=0.7
        ).add_to(m)

# Add legend
legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 150px; height: 140px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:12px; padding: 10px">
    <h5>Activity Types</h5>
    <i class="fa fa-circle" style="color:red"></i> MECH_HFR<br>
    <i class="fa fa-circle" style="color:orange"></i> BENEFICIAL_FIRE<br>
    <i class="fa fa-circle" style="color:gray"></i> NOT_DEFINED<br>
    <i class="fa fa-circle" style="color:green"></i> GRAZING<br>
    <i class="fa fa-circle" style="color:darkgreen"></i> TREE_PLNTING<br>
    <i class="fa fa-circle" style="color:blue"></i> WATSHD_IMPRV
    </div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

display(m)

## Activity Quantity by Admin Organizations

In [ ]:
# Calculate total activity quantity by admin organization
admin_quantity = ifprs.groupby('ADMIN_ORG_NAME')['ACTIVITY_QUANTITY'].agg(['sum', 'count', 'mean']).round(2)
admin_quantity.columns = ['Total Quantity', 'Number of Activities', 'Average Quantity']
admin_quantity = admin_quantity.sort_values('Total Quantity', ascending=False)

print("Activity Quantity by Admin Organization:")
display(admin_quantity)

# Create bar chart
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Total quantity bar chart
admin_quantity['Total Quantity'].plot(kind='bar', ax=ax1, color='skyblue')
ax1.set_title('Total Activity Quantity by Admin Organization', fontsize=14, fontweight='bold')
ax1.set_xlabel('Admin Organization')
ax1.set_ylabel('Total Activity Quantity')
ax1.tick_params(axis='x', rotation=45)

# Number of activities bar chart
admin_quantity['Number of Activities'].plot(kind='bar', ax=ax2, color='lightcoral')
ax2.set_title('Number of Activities by Admin Organization', fontsize=14, fontweight='bold')
ax2.set_xlabel('Admin Organization')
ax2.set_ylabel('Number of Activities')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Activity Quantity by Activity Types

In [ ]:
# Calculate total activity quantity by activity category
activity_quantity = ifprs.groupby('ACTIVITY_CAT')['ACTIVITY_QUANTITY'].agg(['sum', 'count', 'mean']).round(2)
activity_quantity.columns = ['Total Quantity', 'Number of Activities', 'Average Quantity']
activity_quantity = activity_quantity.sort_values('Total Quantity', ascending=False)

print("Activity Quantity by Activity Type:")
display(activity_quantity)

# Create visualizations
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Total quantity bar chart
activity_quantity['Total Quantity'].plot(kind='bar', ax=ax1, color='gold')
ax1.set_title('Total Activity Quantity by Activity Type', fontsize=14, fontweight='bold')
ax1.set_xlabel('Activity Category')
ax1.set_ylabel('Total Activity Quantity')
ax1.tick_params(axis='x', rotation=45)

# Number of activities bar chart
activity_quantity['Number of Activities'].plot(kind='bar', ax=ax2, color='lightblue')
ax2.set_title('Number of Activities by Type', fontsize=14, fontweight='bold')
ax2.set_xlabel('Activity Category')
ax2.set_ylabel('Number of Activities')
ax2.tick_params(axis='x', rotation=45)

# Pie chart for number of activities
activity_quantity['Number of Activities'].plot(kind='pie', ax=ax3, autopct='%1.1f%%', startangle=90)
ax3.set_title('Distribution of Activities by Type', fontsize=14, fontweight='bold')
ax3.set_ylabel('')

# Average quantity bar chart
activity_quantity['Average Quantity'].plot(kind='bar', ax=ax4, color='lightgreen')
ax4.set_title('Average Activity Quantity by Type', fontsize=14, fontweight='bold')
ax4.set_xlabel('Activity Category')
ax4.set_ylabel('Average Quantity')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Activity Quantity by Vegetation Types

In [ ]:
# Calculate total activity quantity by vegetation type
veg_quantity = ifprs.groupby('BROAD_VEGETATION_TYPE')['ACTIVITY_QUANTITY'].agg(['sum', 'count', 'mean']).round(2)
veg_quantity.columns = ['Total Quantity', 'Number of Activities', 'Average Quantity']
veg_quantity = veg_quantity.sort_values('Total Quantity', ascending=False)

print("Activity Quantity by Vegetation Type:")
display(veg_quantity)

# Create visualizations
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Total quantity bar chart
veg_quantity['Total Quantity'].plot(kind='bar', ax=ax1, color='forestgreen')
ax1.set_title('Total Activity Quantity by Vegetation Type', fontsize=14, fontweight='bold')
ax1.set_xlabel('Vegetation Type')
ax1.set_ylabel('Total Activity Quantity')
ax1.tick_params(axis='x', rotation=45)

# Number of activities bar chart
veg_quantity['Number of Activities'].plot(kind='bar', ax=ax2, color='sandybrown')
ax2.set_title('Number of Activities by Vegetation Type', fontsize=14, fontweight='bold')
ax2.set_xlabel('Vegetation Type')
ax2.set_ylabel('Number of Activities')
ax2.tick_params(axis='x', rotation=45)

# Pie chart for number of activities
veg_quantity['Number of Activities'].plot(kind='pie', ax=ax3, autopct='%1.1f%%', startangle=90)
ax3.set_title('Distribution of Activities by Vegetation Type', fontsize=14, fontweight='bold')
ax3.set_ylabel('')

# Average quantity bar chart
veg_quantity['Average Quantity'].plot(kind='bar', ax=ax4, color='mediumpurple')
ax4.set_title('Average Activity Quantity by Vegetation Type', fontsize=14, fontweight='bold')
ax4.set_xlabel('Vegetation Type')
ax4.set_ylabel('Average Quantity')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Summary Statistics

In [ ]:
# Overall summary statistics
print("=== IFPRS DATA SUMMARY ===")
print(f"Total number of activities: {len(ifprs)}")
print(f"Total activity quantity: {ifprs['ACTIVITY_QUANTITY'].sum():.2f}")
print(f"Average activity quantity: {ifprs['ACTIVITY_QUANTITY'].mean():.2f}")
print(f"Date range: {ifprs['ACTIVITY_START'].min()} to {ifprs['ACTIVITY_END'].max()}")

print("\n=== BY ADMIN ORGANIZATION ===")
for org in ifprs['ADMIN_ORG_NAME'].unique():
    if pd.notna(org):
        org_data = ifprs[ifprs['ADMIN_ORG_NAME'] == org]
        print(f"{org}: {len(org_data)} activities, {org_data['ACTIVITY_QUANTITY'].sum():.2f} total quantity")

print("\n=== BY ACTIVITY TYPE ===")
for activity in ifprs['ACTIVITY_CAT'].unique():
    if pd.notna(activity):
        activity_data = ifprs[ifprs['ACTIVITY_CAT'] == activity]
        print(f"{activity}: {len(activity_data)} activities, {activity_data['ACTIVITY_QUANTITY'].sum():.2f} total quantity")

print("\n=== BY VEGETATION TYPE ===")
for veg in ifprs['BROAD_VEGETATION_TYPE'].unique():
    if pd.notna(veg):
        veg_data = ifprs[ifprs['BROAD_VEGETATION_TYPE'] == veg]
        print(f"{veg}: {len(veg_data)} activities, {veg_data['ACTIVITY_QUANTITY'].sum():.2f} total quantity")